In [1]:
# CELL 1: Imports + quick file check
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import joblib
import difflib
import re
import os

print("Working dir:", os.getcwd())
print("Files in /mnt/data (if present):", os.listdir("/mnt/data") if os.path.exists("/mnt/data") else "no /mnt/data")


Working dir: c:\Users\Jay\Desktop\SE proj\New folder\Back
Files in /mnt/data (if present): no /mnt/data


In [2]:
# CELL 2: Load CSVs (adjust paths if needed)
train = pd.read_csv("Training.csv")
test  = pd.read_csv("Testing.csv")

print("Initial shapes -> train:", train.shape, ", test:", test.shape)
# peek last columns to check for 'Unnamed'
print("Train last cols:", train.columns.tolist()[-6:])
print("Test  last cols:", test.columns.tolist()[-6:])


Initial shapes -> train: (4920, 134) , test: (42, 133)
Train last cols: ['inflammatory_nails', 'blister', 'red_sore_around_nose', 'yellow_crust_ooze', 'prognosis', 'Unnamed: 133']
Test  last cols: ['small_dents_in_nails', 'inflammatory_nails', 'blister', 'red_sore_around_nose', 'yellow_crust_ooze', 'prognosis']


In [3]:
# CELL 3: Clean dataset (drop Unnamed and fully-empty columns)
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]   # drop 'Unnamed' columns
train = train.dropna(axis=1, how='all')                        # drop fully empty cols
test  = test.loc[:, ~test.columns.str.contains('^Unnamed')]
test  = test.dropna(axis=1, how='all')

print("After cleaning -> train:", train.shape, ", test:", test.shape)


After cleaning -> train: (4920, 133) , test: (42, 133)


In [4]:
# CELL 4: Ensure 'prognosis' exists
assert 'prognosis' in train.columns, "'prognosis' column missing from Training.csv"
assert 'prognosis' in test.columns,  "'prognosis' column missing from Testing.csv"


In [5]:
# CELL 5: Align feature columns (exclude 'prognosis'); keep train order
train_features = [c for c in train.columns if c != 'prognosis']
test_features  = [c for c in test.columns if c != 'prognosis']

# use only features common to both files, preserving train order
features = [c for c in train_features if c in test_features]

print("Number of features used:", len(features))
print("Example feature names (first 15):", features[:15])

# Subset X and X_test to same feature set & order
X = train[features].astype(int)   # cast to int (if binary 0/1)
y = train['prognosis'].astype(str)

X_test = test[features].astype(int)
y_test = test['prognosis'].astype(str)

print("X shape:", X.shape, "X_test shape:", X_test.shape)


Number of features used: 132
Example feature names (first 15): ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue']
X shape: (4920, 132) X_test shape: (42, 132)


In [6]:
# CELL 6: Encode labels and train Decision Tree
le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X, y_encoded)

# Save model + encoder
joblib.dump(clf, "model.joblib")
joblib.dump(le, "label_encoder.joblib")
print("Model and encoder saved as model.joblib + label_encoder.joblib")


Model and encoder saved as model.joblib + label_encoder.joblib


In [7]:
# CELL 7: Evaluate carefully (handle unseen labels in test if any)
unseen = set(y_test.unique()) - set(le.classes_)
print("Unseen labels in test (if any):", unseen)

if unseen:
    # Drop rows with unseen labels for evaluation only
    mask = y_test.isin(le.classes_)
    X_test_valid = X_test[mask]
    y_test_valid = y_test[mask]
    y_test_encoded = le.transform(y_test_valid)
    y_pred = clf.predict(X_test_valid)
    acc = accuracy_score(y_test_encoded, y_pred)
    print(f"Accuracy on valid subset (dropped {mask.size - X_test_valid.shape[0]} rows):", acc)
else:
    y_test_encoded = le.transform(y_test)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test_encoded, y_pred)
    print("Accuracy on test set:", acc)


Unseen labels in test (if any): set()
Accuracy on test set: 0.9761904761904762


d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")


In [8]:
# CELL 8: Robust predict function for free-text symptom lists
# It tries exact match, normalized match (spaces->underscore, lower), close-matches via difflib,
# and substring matches as a last resort.

def normalize_symptom(s):
    s = s.lower().strip()
    s = re.sub(r'[^a-z0-9 _-]', '', s)   # remove punctuation
    s = s.replace('-', ' ').replace('  ', ' ')
    s = s.replace(' ', '_')
    return s

# build normalized -> actual feature name map
features_norm = {f.lower().replace(' ', '_'): f for f in features}

def predict_disease(symptoms_list):
    input_vec = [0]*len(features)
    for symptom in symptoms_list:
        n = normalize_symptom(symptom)
        col = None
        # direct normalized name match
        if n in features_norm:
            col = features_norm[n]
        else:
            # close match to normalized names
            candidates = difflib.get_close_matches(n, list(features_norm.keys()), n=1, cutoff=0.6)
            if candidates:
                col = features_norm[candidates[0]]
            else:
                # substring search (looser)
                for k,v in features_norm.items():
                    if n in k or k in n:
                        col = v
                        break
        if col is not None:
            idx = features.index(col)
            input_vec[idx] = 1
        # if col is None, symptom ignored (not found); you can log it if desired
    pred = clf.predict([input_vec])[0]
    disease = le.inverse_transform([pred])[0]
    return disease

# Quick demo: use actual feature names or friendly phrases
print("Demo using first 4 real features:", features[:4])
print("Predicted disease for those features:", predict_disease(features[:4]))
# or using free-text
print("Predicted (free-text example):", predict_disease(["itching", "skin rash", "nodal skin eruptions"]))


Demo using first 4 real features: ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing']
Predicted disease for those features: Fungal infection
Predicted (free-text example): Fungal infection


d:\New folder (3)\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
d:\New folder (3)\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [9]:
# Notebook cell
from pathlib import Path
print("CSV files in working dir:")
for f in Path(".").glob("*.csv"):
    print(" -", f.name)

import pandas as pd

# quick load (won't error if not present): adjust names if your files are different
train_path = Path("Training.csv")
test_path  = Path("Testing.csv")
meds_path  = Path("cleaned_medicines.csv")

# load if present
train = pd.read_csv(train_path) if train_path.exists() else None
test  = pd.read_csv(test_path)  if test_path.exists()  else None
meds  = pd.read_csv(meds_path)  if meds_path.exists()  else None

print("train loaded:", train is not None, "test loaded:", test is not None, "meds loaded:", meds is not None)

# inspect columns and top rows (if available)
if train is not None:
    display(train.shape)
    display(train.head())
    print("Columns:", train.columns.tolist())

if test is not None:
    display(test.shape)
    display(test.head())
    print("Columns:", test.columns.tolist())

if meds is not None:
    display(meds.shape)
    display(meds.head())
    print("Columns:", meds.columns.tolist())


CSV files in working dir:
 - classification_report.csv
 - cleaned_medicines.csv
 - Testing.csv
 - Training.csv


train loaded: True test loaded: True meds loaded: True


(4920, 134)

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  scurring  \
0       0           0             0        0                 0  ...         0   
1       0           0             0        0                 0  ...         0   
2       0           0             0        0                 0  ...         0   
3       0           0             0        0                 0  ...         0   
4       0           0             0        0                 0  ...         0   

   skin_peeling  silver_like_dusting  small_dents_in_nails  \
0             0                    0                     0   
1             0                    0                     0   
2             0                    0                     0   
3             0                    0                     0   
4             0                    0                     0   

   inflammatory_nails  blister  red_sore_around_nose  yellow_crust_ooze  \
0                   0        0                     0                  0   
1                   0        0                     0                  0   
2                   0        0                     0                  0   
3                   0        0                     0                  0   
4                   0        0                     0                  0   

          prognosis  Unnamed: 133  
0  Fungal infection           NaN  
1  Fungal infection           NaN  
2  Fungal infection           NaN  
3  Fungal infection           NaN  
4  Fungal infection           NaN  

[5 rows x 134 columns]

Columns: ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs', 'fast_heart_rate', 'pain_d

...

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          0                     0                    1          1   
2        0          0                     0                    0          0   
3        1          0                     0                    0          0   
4        1          1                     0                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  \
0       0           0             0        0                 0  ...   
1       1           0             0        0                 0  ...   
2       0           0             1        1                 1  ...   
3       0           0             0        0                 0  ...   
4       0           0             1        0                 0  ...   

   blackheads  scurring  skin_peeling  silver_like_dusting  \
0           0         0             0                    0   
1           0         0             0                    0   
2           0         0             0                    0   
3           0         0             0                    0   
4           0         0             0                    0   

   small_dents_in_nails  inflammatory_nails  blister  red_sore_around_nose  \
0                     0                   0        0                     0   
1                     0                   0        0                     0   
2                     0                   0        0                     0   
3                     0                   0        0                     0   
4                     0                   0        0                     0   

   yellow_crust_ooze            prognosis  
0                  0     Fungal infection  
1                  0              Allergy  
2                  0                 GERD  
3                  0  Chronic cholestasis  
4                  0        Drug Reaction  

[5 rows x 133 columns]

Columns: ['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing', 'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting', 'vomiting', 'burning_micturition', 'spotting_ urination', 'fatigue', 'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings', 'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat', 'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes', 'breathlessness', 'sweating', 'dehydration', 'indigestion', 'headache', 'yellowish_skin', 'dark_urine', 'nausea', 'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain', 'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever', 'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure', 'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes', 'malaise', 'blurred_and_distorted_vision', 'phlegm', 'throat_irritation', 'redness_of_eyes', 'sinus_pressure', 'runny_nose', 'congestion', 'chest_pain', 'weakness_in_limbs', 'fast_heart_rate', 'pain_d

(23939, 5)

disease_name                      med_name  \
0     ADHD (7)       Atrest 25mg Tablet 10'S   
1     ADHD (7)          Capnea Injection 1ml   
2     ADHD (7)      Capnea Oral Solution 1ml   
3     ADHD (7)  Cognistar 30mg Injection 1'S   
4     ADHD (7)  Cognistar 60mg Injection 1's   

                                     generic_name  \
0               Generic Name  Tetrabenazine 25 mg   
1                    Generic Name  Caffeine 20 mg   
2                    Generic Name  Caffeine 20 mg   
3  Generic Name  Cerebroprotein Hydrolysate 30 mg   
4  Generic Name  Cerebroprotein Hydrolysate 60 mg   

                                            img_urls  \
0  https://www.netmeds.com/images/product-v1/600x...   
1  https://www.netmeds.com/images/product-v1/600x...   
2  https://www.netmeds.com/images/product-v1/600x...   
3  https://www.netmeds.com/images/product-v1/600x...   
4  https://www.netmeds.com/images/product-v1/600x...   

                                             med_url  
0  https://www.netmeds.com/prescriptions/atrest-2...  
1  https://www.netmeds.com/prescriptions/capnea-i...  
2  https://www.netmeds.com/prescriptions/capnea-s...  
3  https://www.netmeds.com/prescriptions/cognista...  
4  https://www.netmeds.com/prescriptions/cognista...

Columns: ['disease_name', 'med_name', 'generic_name', 'img_urls', 'med_url']


In [10]:
# Notebook cell: auto-detect symptom columns and label column
import re

def detect_columns(df):
    cols = df.columns.tolist()
    # find symptom columns (names that contain 'symptom' or numeric pattern)
    symptom_cols = [c for c in cols if re.search(r"symptom", c, re.I)]
    # fallback: look for many columns with few unique values (heuristic)
    if not symptom_cols:
        maybe = [c for c in cols if df[c].nunique() < df.shape[0] and df[c].dtype == object]
        # pick first 10 heuristics that seem like symptoms (this is heuristic; inspect manually)
        symptom_cols = maybe[:17]
    # detect label column: common names
    label_candidates = [c for c in cols if c.lower() in ("prognosis","disease","diagnosis","target","label")]
    label_col = label_candidates[0] if label_candidates else cols[-1]  # fallback to last column
    return symptom_cols, label_col

if train is not None:
    symptom_cols, label_col = detect_columns(train)
    print("Detected symptom columns count:", len(symptom_cols))
    print("Sample symptom columns:", symptom_cols[:10])
    print("Detected label column:", label_col)
else:
    print("Training.csv not present — please upload it.")


Detected symptom columns count: 1
Sample symptom columns: ['prognosis']
Detected label column: prognosis


In [11]:
# Notebook cell
import numpy as np
import json
import re

def normalize_symptom(s):
    if not isinstance(s, str): 
        return ""
    s = s.strip().lower()
    s = re.sub(r"[^a-z0-9\s\-]", " ", s)   # remove punctuation
    s = re.sub(r"\s+", " ", s)
    s = s.replace(" ", "_")
    return s

# Example:
print(normalize_symptom("  High Fever  "))
print(normalize_symptom("Sore-throat"))


high_fever
sore-throat


In [12]:
# Notebook cell: adapt SYMPTOM_COLS and LABEL_COL if auto-detect was wrong
if train is None:
    raise SystemExit("Upload Training.csv and re-run cells.")

SYMPTOM_COLS, LABEL_COL = detect_columns(train)
print("Using", len(SYMPTOM_COLS), "symptom columns.")

# normalize all symptom columns
for c in SYMPTOM_COLS:
    train[c] = train[c].fillna("").astype(str).apply(normalize_symptom)
    test[c]  = test[c].fillna("").astype(str).apply(normalize_symptom) if test is not None else train[c]

# build vocabulary: unique symptom tokens seen in training
vocab = set()
for c in SYMPTOM_COLS:
    vocab.update(s for s in train[c].unique() if s and s != "nan")
vocab = sorted(vocab)
print("Symptom vocab size:", len(vocab))
# save symptom_list for later use
with open("symptom_list.json", "w") as f:
    json.dump(vocab, f, indent=2)

vocab_index = {s:i for i,s in enumerate(vocab)}

def rows_to_multihot(df, vocab_index):
    X = np.zeros((len(df), len(vocab_index)), dtype=int)
    for i, row in df.iterrows():
        for c in SYMPTOM_COLS:
            s = row[c]
            if s and s in vocab_index:
                X[i, vocab_index[s]] = 1
    return X

X_train = rows_to_multihot(train, vocab_index)
y_train_raw = train[LABEL_COL].astype(str).str.strip()

if test is not None:
    for c in SYMPTOM_COLS:
        test[c] = test[c].fillna("").astype(str).apply(normalize_symptom)
    X_test = rows_to_multihot(test, vocab_index)
    y_test_raw = test[LABEL_COL].astype(str).str.strip()
else:
    X_test = None
    y_test_raw = None

print("X_train shape:", X_train.shape)
if X_test is not None:
    print("X_test shape:", X_test.shape)


Using 1 symptom columns.
Symptom vocab size: 41
X_train shape: (4920, 41)
X_test shape: (42, 41)


In [13]:
# Notebook cell
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train_raw)
if y_test_raw is not None:
    y_test = le.transform(y_test_raw)  # assumes test labels exist in training label set

# Train classifier
clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Evaluate
if X_test is not None:
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test accuracy:", acc)
    print(classification_report(y_test, y_pred, target_names=le.classes_))
  # truncated if many classes
else:
    print("No Testing.csv provided — trained on Training.csv only.")

# Save artifacts
joblib.dump(clf, "model.joblib")
joblib.dump(le,  "label_encoder.joblib")
print("Saved: model.joblib, label_encoder.joblib, symptom_list.json")


Test accuracy: 1.0
                                       precision    recall  f1-score   support

_vertigo_paroymsal_positional_vertigo       1.00      1.00      1.00         1
                                 acne       1.00      1.00      1.00         1
                                 aids       1.00      1.00      1.00         1
                  alcoholic_hepatitis       1.00      1.00      1.00         1
                              allergy       1.00      1.00      1.00         1
                            arthritis       1.00      1.00      1.00         1
                     bronchial_asthma       1.00      1.00      1.00         1
                 cervical_spondylosis       1.00      1.00      1.00         1
                          chicken_pox       1.00      1.00      1.00         1
                  chronic_cholestasis       1.00      1.00      1.00         1
                          common_cold       1.00      1.00      1.00         1
                               d

d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
d:\New folder (3)\Lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is g

In [14]:
# Notebook cell
# Inspect meds dataframe and create disease -> [meds] mapping
if meds is None:
    print("cleaned_medicines.csv not found; please upload.")
else:
    display(meds.head())
    print("Columns:", meds.columns.tolist())

    # common column names to look for
    potential_disease_cols = [c for c in meds.columns if c.lower() in ("disease","prognosis","condition","illness")]
    potential_med_cols = [c for c in meds.columns if "med" in c.lower() or "drug" in c.lower() or "medicine" in c.lower()]

    print("Detected disease cols:", potential_disease_cols)
    print("Detected medicine cols:", potential_med_cols)

    # If automatic detection didn't find clear names, update these variables manually:
    disease_col = potential_disease_cols[0] if potential_disease_cols else meds.columns[0]
    med_col     = potential_med_cols[0]     if potential_med_cols else meds.columns[1] if len(meds.columns)>1 else meds.columns[-1]

    # Normalize disease names and medication names
    meds[disease_col] = meds[disease_col].astype(str).str.strip()
    meds[med_col]     = meds[med_col].astype(str).str.strip()

    # Group meds by disease -> unique list
    med_map = meds.groupby(disease_col)[med_col].apply(lambda s: sorted(set(s.dropna().astype(str)))).to_dict()

    # Save to JSON
    import json
    with open("medications.json", "w") as f:
        json.dump(med_map, f, indent=2)

    print("Saved medications.json with", len(med_map), "diseases.")
    # show a tiny sample
    k = list(med_map.keys())[:5]
    for d in k:
        print("-", d, "->", med_map[d][:5])


disease_name                      med_name  \
0     ADHD (7)       Atrest 25mg Tablet 10'S   
1     ADHD (7)          Capnea Injection 1ml   
2     ADHD (7)      Capnea Oral Solution 1ml   
3     ADHD (7)  Cognistar 30mg Injection 1'S   
4     ADHD (7)  Cognistar 60mg Injection 1's   

                                     generic_name  \
0               Generic Name  Tetrabenazine 25 mg   
1                    Generic Name  Caffeine 20 mg   
2                    Generic Name  Caffeine 20 mg   
3  Generic Name  Cerebroprotein Hydrolysate 30 mg   
4  Generic Name  Cerebroprotein Hydrolysate 60 mg   

                                            img_urls  \
0  https://www.netmeds.com/images/product-v1/600x...   
1  https://www.netmeds.com/images/product-v1/600x...   
2  https://www.netmeds.com/images/product-v1/600x...   
3  https://www.netmeds.com/images/product-v1/600x...   
4  https://www.netmeds.com/images/product-v1/600x...   

                                             med_url  
0  https://www.netmeds.com/prescriptions/atrest-2...  
1  https://www.netmeds.com/prescriptions/capnea-i...  
2  https://www.netmeds.com/prescriptions/capnea-s...  
3  https://www.netmeds.com/prescriptions/cognista...  
4  https://www.netmeds.com/prescriptions/cognista...

Columns: ['disease_name', 'med_name', 'generic_name', 'img_urls', 'med_url']
Detected disease cols: []
Detected medicine cols: ['med_name', 'med_url']
Saved medications.json with 141 diseases.
- ADHD (7) -> ["Atrest 25mg Tablet 10'S", 'Capnea Injection 1ml', 'Capnea Oral Solution 1ml', "Cognistar 30mg Injection 1'S", "Cognistar 60mg Injection 1's"]
- Acne (358) -> ['A Ret 0.025% Gel 20gm', 'A Ret 0.05% Gel 20gm', 'A Ret 0.1% Gel 20gm', 'AB NEXT Gel 20gm', 'ABAKA Gel 20gm']
- Alcohol Addiction (14) -> ["ACAMPROVAC 333 Tablet 6's", "ADDTREX 50 Tablet 10's", "ALCONIX 500mg Tablet 4's", "Acamprol 333mg Tablet 6'S", "Acamptas 333mg Tablet 6'S"]
- Allergies (756) -> ["1 AL 10mg Tablet 15'S", "1 AL 5mg Tablet 10'S", '1 AL Syrup 30ml', "A FYL 200 SR Tablet 10's", "ADDCET L Tablet 10's"]
- Alzheimer (87) -> ["ALZIL M FORTE Tablet 10's", "ARICEP M FORTE Tablet 10's", "Admenta 10mg Tablet 10'S", "Admenta 5mg Tablet 10'S", "Almerzil 10mg Tablet 10'S"]


In [15]:
# Notebook cell: inference helper
import json
from rapidfuzz import fuzz
import numpy as np

# load artifacts
with open("symptom_list.json") as f:
    SYMPTOMS = json.load(f)
clf = joblib.load("model.joblib")
le  = joblib.load("label_encoder.joblib")
with open("medications.json") as f:
    MED_MAP = json.load(f)

sym2idx = {s:i for i,s in enumerate(SYMPTOMS)}

def normalize_text(s):
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9 _\-]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    s = s.replace(" ", "_")
    return s

def extract_symptoms_from_text(text, threshold=75):
    text_n = normalize_text(text)
    matches = set()
    # direct contains
    for s in SYMPTOMS:
        if s in text_n:
            matches.add(s)
    # fuzzy
    for s in SYMPTOMS:
        if fuzz.partial_ratio(s, text_n) >= threshold:
            matches.add(s)
    # prune: keep longer specific terms
    pruned = set()
    for s in sorted(matches, key=len, reverse=True):
        if not any((s != t and s in t) for t in pruned):
            pruned.add(s)
    return sorted(pruned)

def symptoms_to_vector(symptoms_list):
    vec = np.zeros(len(SYMPTOMS), dtype=int)
    for s in symptoms_list:
        if s in sym2idx:
            vec[sym2idx[s]] = 1
    return vec.reshape(1, -1)

def predict_from_text(text, topn=3):
    matched = extract_symptoms_from_text(text)
    if not matched:
        return {"matched_symptoms": [], "predictions": [], "medications": []}
    X = symptoms_to_vector(matched)
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X)[0]
        top_idx = np.argsort(proba)[::-1][:topn]
        preds = [{"disease": le.inverse_transform([i])[0], "confidence": float(proba[i])} for i in top_idx]
    else:
        p = clf.predict(X)[0]
        preds = [{"disease": le.inverse_transform([p])[0], "confidence": None}]
    # medications for top1
    top1 = preds[0]["disease"]
    meds_for_top1 = MED_MAP.get(top1, [])
    return {"matched_symptoms": matched, "predictions": preds, "medications": meds_for_top1}

# quick demo
print(predict_from_text("I have high fever and severe headache since morning"))


{'matched_symptoms': ['acne'], 'predictions': [{'disease': 'acne', 'confidence': 1.0}, {'disease': 'urinary_tract_infection', 'confidence': 0.0}, {'disease': 'typhoid', 'confidence': 0.0}], 'medications': []}


In [16]:
tests = [
    "I have fever, cough and body ache",
    "severe headache and blurred vision",
    "stomach pain and vomiting",
    "runny nose and sneezing"
]

for t in tests:
    r = predict_from_text(t)
    print("===\nInput:", t)
    print("Matched symptoms:", r["matched_symptoms"])
    print("Top predictions:", r["predictions"])
    print("Suggested meds:", r["medications"][:8])


===
Input: I have fever, cough and body ache
Matched symptoms: ['acne']
Top predictions: [{'disease': 'acne', 'confidence': 1.0}, {'disease': 'urinary_tract_infection', 'confidence': 0.0}, {'disease': 'typhoid', 'confidence': 0.0}]
Suggested meds: []
===
Input: severe headache and blurred vision
Matched symptoms: ['acne']
Top predictions: [{'disease': 'acne', 'confidence': 1.0}, {'disease': 'urinary_tract_infection', 'confidence': 0.0}, {'disease': 'typhoid', 'confidence': 0.0}]
Suggested meds: []
===
Input: stomach pain and vomiting
Matched symptoms: []
Top predictions: []
Suggested meds: []
===
Input: runny nose and sneezing
Matched symptoms: []
Top predictions: []
Suggested meds: []


In [17]:
# (optional) re-save artifacts explicitly
import joblib, json
joblib.dump(clf, "model.joblib")
joblib.dump(le, "label_encoder.joblib")
with open("symptom_list.json", "w") as f:
    json.dump(SYMPTOMS, f, indent=2)
with open("medications.json", "w") as f:
    json.dump(MED_MAP, f, indent=2)
print("Artifacts saved.")


Artifacts saved.


In [19]:
from flask import Flask, request, jsonify
import joblib, json
import numpy as np

# Initialize Flask
app = Flask(__name__)

# Load trained artifacts
model = joblib.load("model.joblib")
label_encoder = joblib.load("label_encoder.joblib")
with open("symptom_list.json", "r") as f:
    symptom_list = json.load(f)
with open("medications.json", "r") as f:
    medications = json.load(f)

# Helper: convert input symptoms to vector
def symptoms_to_vector(user_symptoms):
    vector = [0] * len(symptom_list)
    for s in user_symptoms:
        if s in symptom_list:
            vector[symptom_list.index(s)] = 1
    return np.array(vector).reshape(1, -1)

# Health check route
@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "Flask API is running. Use POST /predict"}), 200

@app.route("/predict", methods=["POST"])
def predict():
    data = request.get_json()
    symptoms = data.get("symptoms", [])

    if not symptoms:
        return jsonify({"error": "No symptoms provided"}), 400

    # Convert to vector
    X = symptoms_to_vector(symptoms)

    # Predict disease
    pred = model.predict(X)[0]
    prob = max(model.predict_proba(X)[0])

    disease = label_encoder.inverse_transform([pred])[0]
    meds = medications.get(disease, ["Consult a doctor"])

    return jsonify({
        "disease": disease,
        "confidence": round(float(prob), 2),
        "medications": meds
    })

if __name__ == "__main__":
    # IMPORTANT: use_reloader=False to avoid restart issues
    app.run(host="127.0.0.1", port=5001, debug=True, use_reloader=False)




 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit


In [ ]:
import requests

# Replace with your backend URL
url = "http://127.0.0.1:5001/predict"

# Example symptoms (as if spoken by patient)
data = {
    "symptoms": "fever cough headache"
}

# Send POST request
response = requests.post(url, json=data)

# Print raw response
print("Status Code:", response.status_code)
print("Response JSON:", response.json())


In [20]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import joblib, json, numpy as np

app = Flask(__name__)
CORS(app)  # ✅ allow frontend requests

# Dummy ML loading (replace with your actual model + files)
try:
    model = joblib.load("model.joblib")
    label_encoder = joblib.load("label_encoder.joblib")
    with open("symptom_list.json", "r") as f:
        symptom_list = json.load(f)
    with open("medications.json", "r") as f:
        medications = json.load(f)
except Exception as e:
    print("⚠️ Could not load model files:", e)
    model, label_encoder, symptom_list, medications = None, None, [], {}

def symptoms_to_vector(user_symptoms):
    vector = [0] * len(symptom_list)
    for s in user_symptoms:
        if s in symptom_list:
            vector[symptom_list.index(s)] = 1
    return np.array(vector).reshape(1, -1)

@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "✅ Flask API running. Use POST /predict"}), 200

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    print("📥 Received data:", data)  # ✅ Debug log

    symptoms = data.get("symptoms", [])
    if not symptoms:
        return jsonify({"error": "No symptoms provided"}), 400

    try:
        # Dummy prediction for testing
        disease = "Flu"
        confidence = 0.85
        meds = ["Paracetamol", "Rest", "Hydration"]

        return jsonify({
            "disease": disease,
            "confidence": confidence,
            "medications": meds
        })
    except Exception as e:
        print("❌ Backend error:", e)
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    # 🔑 Important fix: use_reloader=False avoids "SystemExit:1"
    # Explicitly set port 5001
    app.run(host="127.0.0.1", port=5001, debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [04/Sep/2025 09:12:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2025 09:12:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Sep/2025 09:13:18] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2025 09:13:18] "POST /predict HTTP/1.1" 200 -


📥 Received data: {'symptoms': ['i', 'have', 'a', 'fever.']}


127.0.0.1 - - [04/Sep/2025 09:13:30] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2025 09:13:30] "POST /predict HTTP/1.1" 200 -


📥 Received data: {'symptoms': ['i', 'have', 'loose', 'motion.']}


127.0.0.1 - - [04/Sep/2025 09:14:49] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Sep/2025 09:14:49] "POST /predict HTTP/1.1" 200 -


📥 Received data: {'symptoms': ['i', 'am', 'a', 'disco', 'dancer.']}
